### Entanglement-based Quantum Key Distribution in a $n$ x $n$ grid topology using Netsquid

#### The objetive of this project is to use the Ekert Protocol in order to implement QKD in a $n$ x $n$ Quantum Network. The following list describes the milestones to be achieved:
* Entangle a random route using Entanglement Swapping (Completed)
* Entangle two or more disjunctive routes (Completed)
* Generate the shared key in each node (In process)  

#### Code functionality: This implementation creates a $n$ x $n$ grid Quantum Network. After the Quantum Network is defined, the simulation starts dynamic protocols depending on the connection requirements. When the simulation ends, it brings back the path, fidelity and time information.

_refs_: 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html 
* Evan Sutcliffe, Matty J. Hoban & Alejandra Beghelli - Multipath Routing for Multipartite State Distribution in Quantum Networks  

_(adapted by D-Cryp7 for Netsquid 1.1.6)_

#### Limitations:
* Conjunctive routes causes a Race Condition error. For now, our aim focus on disjunctive routes, due to the possible alternative options.

In [ ]:
# Library imports

import pandas
import pydynaa
import numpy as np
from random import randint

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Util.number import long_to_bytes
from hashlib import sha256

import netsquid as ns
from netsquid.qubits import ketstates as ks
from netsquid.components import Message, QuantumProcessor, QuantumProgram, PhysicalInstruction
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel, QuantumErrorModel
from netsquid.components.instructions import INSTR_MEASURE_BELL, INSTR_X, INSTR_Z
from netsquid.nodes import Node, Network
from netsquid.protocols import LocalProtocol, NodeProtocol, Signals
from netsquid.util.datacollector import DataCollector
from netsquid.examples.teleportation import EntanglingConnection, ClassicalConnection

In [ ]:
class FibreDepolarizeModel(QuantumErrorModel):
    """
    Custom non-physical error model used to show the effectiveness
    of Entanglement Swapping.

    The default values are chosen to make a nice figure,
    and don't represent any physical system.

    Parameters
    ----------
    p_depol_init : float, optional
        Probability of depolarization on entering a fibre.
        Must be between 0 and 1. Default 0.009
    p_depol_length : float, optional
        Probability of depolarization per km of fibre.
        Must be between 0 and 1. Default 0.025

    """

    def __init__(self, p_depol_init = 0.009, p_depol_length = 0.025):
        super().__init__()
        self.properties['p_depol_init'] = p_depol_init
        self.properties['p_depol_length'] = p_depol_length
        self.required_properties = ['length']

    def error_operation(self, qubits, delta_time = 0, **kwargs):
        """
        Uses the length property to calculate a depolarization probability,
        and applies it to the qubits.

        Parameters
        ----------
        qubits : tuple of :obj:`~netsquid.qubits.qubit.Qubit`
            Qubits to apply noise to.
        delta_time : float, optional
            Time qubits have spent on a component [ns]. Not used.

        """
        for qubit in qubits:
            prob = 1 - (1 - self.properties['p_depol_init']) * np.power(
                10, - kwargs['length'] ** 2 * self.properties['p_depol_length'] / 10)
            ns.qubits.depolarize(qubit, prob = prob)

In [ ]:
class SwapCorrectProgram(QuantumProgram):
    """
    Quantum processor program that applies all swap corrections.
    """
    
    default_num_qubits = 1

    def set_corrections(self, x_corr, z_corr):
        self.x_corr = x_corr % 2
        self.z_corr = z_corr % 2

    def program(self):
        q1, = self.get_qubit_indices(1)
        if self.x_corr == 1:
            self.apply(INSTR_X, q1)
        if self.z_corr == 1:
            self.apply(INSTR_Z, q1)
        yield self.run()

In [ ]:
def create_qprocessor(name):
    """
    Factory to create a quantum processor for each node in the Quantum Network.

    Has four memory positions and the physical instructions necessary for teleportation.

    Parameters
    ----------
    name : str
        Name of the quantum processor.

    Returns
    -------
    :class:`~netsquid.components.qprocessor.QuantumProcessor`
        A quantum processor to specification.

    """
    noise_rate = 200
    gate_duration = 1
    gate_noise_model = DephaseNoiseModel(noise_rate)
    mem_noise_model = DepolarNoiseModel(noise_rate)
    physical_instructions = [
        PhysicalInstruction(INSTR_X, duration=gate_duration,
                            quantum_noise_model=gate_noise_model),
        PhysicalInstruction(INSTR_Z, duration=gate_duration,
                            quantum_noise_model=gate_noise_model),
        PhysicalInstruction(INSTR_MEASURE_BELL, duration=gate_duration),
    ]
    qproc = QuantumProcessor(name, num_positions=4, fallback_to_nonphysical=False,
                             mem_noise_models=[mem_noise_model] * 4,
                             phys_instructions=physical_instructions)
    return qproc

In [ ]:
def get_neighbours(network, node, n):
    '''
    Get neighbours of each node in a n x n Grid Quantum Network
    '''
    neighbours = []
    nodes = network.nodes
    pos = eval(node.name)
    i, j = pos[0], pos[1]
    if i + 1 < n:
        neighbours.append(nodes[str((i + 1, j))])
    if i - 1 >= 0:
        neighbours.append(nodes[str((i - 1, j))])
    if j + 1 < n:
        neighbours.append(nodes[str((i, j + 1))])
    if j - 1 >= 0:
        neighbours.append(nodes[str((i, j - 1))])
    return neighbours

In [ ]:
def setup_datacollector(network, protocol, path, node_qconnections):
    """
    Setup the datacollector to calculate the fidelity
    when the CorrectionProtocol has finished.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    protocol : :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Datacollector recording fidelity data.

    """
    def random_basis():
        r = randint(0, 1)
        return ns.Z if r else ns.X

    def measure(q, obs):
        measurement_result, prob = ns.qubits.measure(q, obs)
        if measurement_result == 0:
            state = "|0>" if obs == ns.Z else "|+>"
        else:
            state = "|1>" if obs == ns.Z else "|->"
        return ("Z" if obs == ns.Z else "X"), measurement_result, prob

    # Ensure nodes are ordered in the chain:
    # nodes = [network.nodes[name] for name in sorted(network.nodes.keys())]
    nodes = [network.nodes[str(name)] for name in path]
    A_port = int(node_qconnections[nodes[0].name][nodes[1].name].split("-")[0][-1])
    B_port = int(node_qconnections[nodes[-1].name][nodes[-2].name].split("-")[0][-1])
    nodes[0].shared_secret = []
    nodes[-1].shared_secret = []
    def calc_fidelity(evexpr):
        qubit_a, = nodes[0].qmemory.peek([A_port])
        qubit_b, = nodes[-1].qmemory.peek([B_port])
        fidelity = ns.qubits.fidelity([qubit_a, qubit_b], ks.b00, squared = True)
        measure_a = measure(qubit_a, random_basis())
        measure_b = measure(qubit_b, random_basis())
        if fidelity >= 0.9:
            if measure_a[0] == measure_b[0]:
                nodes[0].shared_secret.append(str(int(measure_a[1])))
                nodes[-1].shared_secret.append(str(int(measure_b[1])))
        return {"fidelity": fidelity, 
                "A_measure": measure_a[0], "A_measure_result": measure_a[1], 
                "B_measure": measure_b[0], "B_measure_result": measure_b[1]}

    dc = DataCollector(calc_fidelity, include_entity_name=False)
    dc.collect_on(pydynaa.EventExpression(source=protocol.subprotocols['CorrectProtocol'],
                                          event_type=Signals.SUCCESS.value))
    return dc

In [ ]:
class SwapProtocol(NodeProtocol):
    """
    Perform Swap on a repeater node.

    Parameters
    ----------
    node : :class:`~netsquid.nodes.node.Node` or None, optional
        Node this protocol runs on.
    name : str
        Name of this protocol.

    """

    def __init__(self, node, name, ccon_R, port_l, port_r):
        super().__init__(node, name)
        self._qmem_input_port_l = self.node.qmemory.ports[port_l]
        self._qmem_input_port_r = self.node.qmemory.ports[port_r]
        self._program = QuantumProgram(num_qubits = 2)
        self.ccon_R = ccon_R
        q1, q2 = self._program.get_qubit_indices(num_qubits = 2)
        self._program.apply(INSTR_MEASURE_BELL, [q1, q2], output_key = "m", inplace = False)

    def run(self):
        while True:
            yield (self.await_port_input(self._qmem_input_port_l) &
                   self.await_port_input(self._qmem_input_port_r))
            # Perform Bell measurement
            yield self.node.qmemory.execute_program(self._program, 
                                                    qubit_mapping=[int(self._qmem_input_port_l.name[-1]), 
                                                                   int(self._qmem_input_port_r.name[-1])])
            m, = self._program.output["m"]
            # Send result to right node on end
            self.node.ports[self.ccon_R].tx_output(Message(m))
            # print(f"Message swapped from {self.ccon_R}: {m}") # for debugging

In [ ]:
class CorrectProtocol(NodeProtocol):
    """
    Perform corrections for a swap on an end-node.

    Parameters
    ----------
    node : :class:`~netsquid.nodes.node.Node` or None, optional
        Node this protocol runs on.
    num_nodes : int
        Number of nodes in the path.

    """

    def __init__(self, node, num_nodes, ccon_L, port):
        super().__init__(node, "CorrectProtocol")
        self._qmem_input_port = self.node.qmemory.ports[port]
        self.num_nodes = num_nodes
        self._x_corr = 0
        self._z_corr = 0
        self._program = SwapCorrectProgram()
        self._counter = 0
        self.ccon_L = ccon_L

    def run(self):
        while True:
            if self.num_nodes > 2:
                yield self.await_port_input(self.node.ports[self.ccon_L])
                message = self.node.ports[self.ccon_L].rx_input()
                if message is None or len(message.items) != 1:
                    continue
                m = message.items[0]
                if m == ks.BellIndex.B01 or m == ks.BellIndex.B11:
                    self._x_corr += 1
                if m == ks.BellIndex.B10 or m == ks.BellIndex.B11:
                    self._z_corr += 1
                self._counter += 1
                if self._counter == self.num_nodes - 2:
                    if self._x_corr or self._z_corr:
                        self._program.set_corrections(self._x_corr, self._z_corr)
                        yield self.node.qmemory.execute_program(self._program, qubit_mapping=[1])
                    self.send_signal(Signals.SUCCESS)
                    self._x_corr = 0
                    self._z_corr = 0
                    self._counter = 0
                    # print(f"Message recieved and corrected from {self.ccon_L}: {m}") # for debugging
            else:
                yield self.await_port_input(self._qmem_input_port)
                self.send_signal(Signals.SUCCESS)
                # print(f"Message recieved from {self.ccon_L}") # for debugging

In [ ]:
def network_setup(n, node_distance, source_frequency):
    
    """
    n x n grid Quantum Network setup.
    - based on Multipath Routing for Multipartite State Distribution in Quantum Networks
    - each node can do Entanglement Swapping

    Parameters:
    - node_distance (float): Distance between nodes [km]
    - source_frecuency (float): Frequency at which the sources create entangled qubits [Hz]

    Returns:
    - class:`~netsquid.nodes.network.Network`
      Network component with all nodes and connections as subcomponents.

    ref: https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html
    """
    network = Network(f"{n}x{n} grid Quantum Network")
    # Create nodes with quantum processors
    nodes = []
    
    node_qconnections = {}
    node_used_ports = {}
    quantum_ports = ["qin" + str(i) for i in range(4)] # 4 memory positions for the QuantumProcessor
    
    for i in range(n):
        for j in range(n):
            nodes.append(Node(f"{i,j}", qmemory = create_qprocessor(f"qproc_{i,j}")))
            node_qconnections[f"({i}, {j})"] = {}
            node_used_ports[f"({i}, {j})"] = []
    network.add_nodes(nodes)
    # Create quantum and classical connections:
    for i in range(len(nodes)):
        current_node = nodes[i]
        neighbours = get_neighbours(network, current_node, n)
        for near in neighbours:
            # print("Establishing a connection: ", current_node.name, "->", near.name) # for debugging
            # Create quantum connection
            try:       
                qconn = EntanglingConnection(name =f"qconn_{current_node.name}<->{near.name}", 
                                             length = node_distance,
                                             source_frequency = source_frequency)
                # Add a noise model which depolarizes the qubits exponentially
                # depending on the connection length
                for channel_name in ['qchannel_C2A', 'qchannel_C2B']:
                    qconn.subcomponents[channel_name].models['quantum_noise_model'] =\
                        FibreDepolarizeModel()
                
                port_name, port_r_name = network.add_connection(
                    current_node, near, connection = qconn, label="quantum")
                
                for l_port in quantum_ports:
                    if l_port not in node_used_ports[current_node.name]:
                        node_used_ports[current_node.name].append(l_port)
                        break
                
                for r_port in quantum_ports:
                    if r_port not in node_used_ports[near.name]:
                        node_used_ports[near.name].append(r_port)
                        break
                        
                node_qconnections[current_node.name][near.name] = str(l_port) + "-" + str(r_port)
                node_qconnections[near.name][current_node.name] = str(r_port) + "-" + str(l_port)
                
                # Forward qconn directly to quantum memories for right and left inputs:
                current_node.ports[port_name].forward_input(current_node.qmemory.ports[str(l_port)])
                near.ports[port_r_name].forward_input(near.qmemory.ports[str(r_port)])
            except:
                pass
            
            # Create classical connection
            cconn = ClassicalConnection(name = f"cconn_{current_node.name}-{near.name}", length = node_distance)
            port_name, port_r_name = network.add_connection(
                current_node, near, connection = cconn, label="classical",
                port_name_node1 = f"ccon_R_{current_node.name}-{near.name}", port_name_node2 = f"ccon_L_{current_node.name}-{near.name}")
            # Forward cconn to right most node
            if f"ccon_L_{current_node.name}-{near.name}" in current_node.ports:
                current_node.ports[f"ccon_L_{current_node.name}-{near.name}"].bind_input_handler(
                    lambda message, _node = current_node: _node.ports[f"ccon_R_{current_node.name}-{near.name}"].tx_output(message))
    return network, node_qconnections

In [ ]:
def setup_repeater_protocol(network, path, node_qconnections):
    """Setup repeater protocol on repeater chain network.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    Returns
    -------
    :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    """
    nodes = [network.nodes[str(name)] for name in path]
    protocol = LocalProtocol(nodes = network.nodes)
    # Add SwapProtocol to all repeater nodes. Note: we use unique names,
    # since the subprotocols would otherwise overwrite each other in the main protocol.
    nodes = [network.nodes[str(name)] for name in path]
    for node in nodes[1:-1]:
        index = path.index(eval(node.name))
        # Specify ccon_R port
        ccon_R = f"ccon_R_{node.name}-{nodes[index + 1].name}"
        port_l = node_qconnections[node.name][nodes[index - 1].name].split("-")[0]
        port_r = node_qconnections[node.name][nodes[index + 1].name].split("-")[0]
        subprotocol = SwapProtocol(node = node, name = f"Swap_{node.name}", ccon_R = ccon_R, port_l = port_l, port_r = port_r)
        protocol.add_subprotocol(subprotocol)
    # Add CorrectProtocol to Bob
    ccon_L = f"ccon_L_{nodes[-2].name}-{nodes[-1].name}"
    # Specify ccon_L port
    port = node_qconnections[nodes[-1].name][nodes[-2].name].split("-")[0]
    subprotocol = CorrectProtocol(nodes[-1], len(nodes), ccon_L, port)
    protocol.add_subprotocol(subprotocol)
    return protocol

In [ ]:
def run_simulation(network, qdf, est_runtime, num_iters, traffic):
    """
    Run the simulation experiment and return the collected data.

    Parameters
    ----------
    num_nodes : int, optional
        Number of nodes in the path
    node_distance : float, optional
        Distance between nodes, larger than 0. Default 20 [km].
    num_iters : int, optional
        Number of simulation runs. Default 100.

    Returns
    -------
    :class:`pandas.DataFrame`
        Dataframe with recorded fidelity data.

    """
    data_collectors = []
    protocols = []
    for path in traffic["path"]:
        protocol = setup_repeater_protocol(network, path, qdf)
        dc = setup_datacollector(network, protocol, path, qdf)
        data_collectors.append(dc)
        protocols.append(protocol)
    for prot in protocols:
        prot.start()
    ns.sim_run(est_runtime * num_iters)
    return data_collectors

In [ ]:
from random import choice
from math import log

def move(a, b):
    if a < b:
        a += 1
    elif a > b:
        a -= 1
    return a

def get_random_route(network):
    network_nodes = []
    for node in network.nodes:
        network_nodes.append(eval(node))
    route = []
    A = choice(network_nodes)
    B = choice(network_nodes)
    while A == B:
        B = choice(network_nodes)
    route.append(A)
    T = [A[0], A[1]]
    while (T[0] != B[0]) or (T[1] != B[1]):
        if T[0] != B[0]:
            T[0] = move(T[0], B[0])
            route.append((T[0], T[1]))
        if T[1] != B[1]:
            T[1] = move(T[1], B[1])
            route.append((T[0], T[1]))
    return route

def binary_entropy(x):
    return -x*log(x, 2) - (1 - x)*log(1 - x, 2)

#### Record Fidelity and Loss metrics (PoC)

In [ ]:
%%time

import time

# Default values

n = 3
node_distance = 20
num_iters = 256
est_runtime = (0.5 + 2 * n - 1) * node_distance * 5e3

results = {
    "path": [],
    "path_length": [],
    "fidelity_rate": [],
    "loss": []
}

try:
    file = open(f"results_{n}x{n}.csv", "r").close()
    print(f"Found results_{n}x{n}.csv")
except:
    print(f"Creating results_{n}x{n}.csv")
    file = open(f"results_{n}x{n}.csv", "w").write("path,path_length,fidelity_rate,loss")

for i in range(10):
    ns.sim_reset()
    network, node_qconnections = network_setup(n, node_distance = node_distance,
                            source_frequency = 1e9 / est_runtime)

    qdf = pandas.DataFrame(node_qconnections)
    qdf = qdf.fillna("")

    path = get_random_route(network)

    traffic = {
        "path": [path]
    }

    print(f"Simulation from path: {path}")

    df = run_simulation(network, qdf, est_runtime, num_iters, traffic)

    rate = df[0].dataframe["fidelity"].sum() / num_iters
    loss = 1 - (df[0].dataframe.shape[0] / num_iters)

    results_df = pandas.read_csv("results_3x3.csv")
    results_df.loc[len(results_df.index)] = [path, len(path), rate * 100, loss * 100]
    results_df.to_csv("results_3x3.csv", index = False)

#### Application: Generate a shared secret between two nodes (QKD)

In [34]:
%%time

import time

# Default values

n = 3
node_distance = 20
num_iters = 256
est_runtime = (0.5 + 2 * n - 1) * node_distance * 5e3


ns.sim_reset()
network, node_qconnections = network_setup(n, node_distance = node_distance,
                        source_frequency = 1e9 / est_runtime)

qdf = pandas.DataFrame(node_qconnections)
qdf = qdf.fillna("")

path = get_random_route(network)

traffic = {
    "path": [path]
}

print(f"Simulation from path: {path}")

df = run_simulation(network, qdf, est_runtime, num_iters, traffic)

assert network.nodes[str(path[0])].shared_secret == network.nodes[str(path[-1])].shared_secret

network.nodes[str(path[0])].shared_secret = sha256(long_to_bytes(int(''.join(network.nodes[str(path[0])].shared_secret), 2))).digest()
network.nodes[str(path[-1])].shared_secret = sha256(long_to_bytes(int(''.join(network.nodes[str(path[-1])].shared_secret), 2))).digest()

data = b"H3xTEL{Qu4nTuM_k3Y_D1s7r1buT10n!}"
cipher = AES.new(network.nodes[str(path[0])].shared_secret, AES.MODE_ECB)
encrypted_data = cipher.encrypt(pad(data, 16))
print(f"Alice encrypted data: {encrypted_data}")

cipher = AES.new(network.nodes[str(path[-1])].shared_secret, AES.MODE_ECB)
decrypted_data = unpad(cipher.decrypt(encrypted_data), 16)
print(f"Bob decrypted data: {decrypted_data}")

Simulation from path: [(0, 1), (1, 1), (1, 0), (2, 0)]
Alice encrypted data: b'c\x8b\xea\xa9\xd6\xbe\x06$\xa8C\xe8\xa0\xca|5o\xf7\xd3U\x8eP\xa0\xd5\x0124\xe5f\x95z\xe7M\xe5\xbbA\x02\x11\xe5\n\x7f\xcd\x97\x90\xc2G\t\x7fD'
Bob decrypted data: b'H3xTEL{Qu4nTuM_k3Y_D1s7r1buT10n!}'
CPU times: user 5.22 s, sys: 82 ms, total: 5.3 s
Wall time: 5.62 s


##### Work In Progress (WIP): Analyse the secret key rate (SKR)

In [29]:
dataframe = df[0].dataframe
fid_df = dataframe[dataframe["fidelity"] >= 0.9]
Z_fid = fid_df[fid_df["B_measure"] == "Z"]
X_fid = fid_df[fid_df["B_measure"] == "X"]
Q_z = Z_fid[Z_fid["A_measure"] != Z_fid["B_measure"]].shape[0] / Z_fid.shape[0]
Q_x = X_fid[X_fid["A_measure"] != X_fid["B_measure"]].shape[0] / X_fid.shape[0]
print(f"QBER in Z basis: {Q_z}")
print(f"QBER in X basis: {Q_x}")
1 - binary_entropy(Q_x) - binary_entropy(Q_z) # some part of the SKR

QBER in Z basis: 0.5161290322580645
QBER in X basis: 0.25


-0.8105273724547893